# Building A Spam Filter With Naive bayes

We will teach the computer to classify spam or non spam messages based on the probability theory - Naive bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo

Lets read the dataset. 

In [2]:
import pandas as pd

sms_spam = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

print(sms_spam.shape)
sms_spam.head()



(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
sms_spam['Label'].value_counts(normalize=True)


ham     0.865937
spam    0.134063
Name: Label, dtype: float64

Around 87% sms are 'ham' and 13% are spam. Which is quite representative of the real life situation.

## Training The Dataset

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

The training set will have 4,458 messages (about 80% of the dataset).
The test set will have 1,114 messages (about 20% of the dataset).

To better understand the purpose of putting a test set aside, let's begin by observing that all 1,114 messages in our test set are already classified by a human.

For this project, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than **80% of the new messages will be classified correctly as spam or ham (non-spam).**

We're going to start by randomizing the entire dataset to ensure that spam and ham messages are spread properly throughout the dataset.



In [4]:
# Randomize the dataset
data_randomized = sms_spam.sample(frac=1, random_state=1)

# Creating index for 80% training set 
training_test_index = round(len(data_randomized) * 0.8)

# using that index, we will split train set and rest of the set will be test set
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


Now this is important that we get the same representation (87% spam and 13% ham) of the enitire dataset in our test and train set. So we need to check that. 

In [5]:
training_set['Label'].value_counts(normalize=True)


ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [6]:
test_set['Label'].value_counts(normalize= True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

Yep, got it right !

## Letter Case And Punctuation

We will begin our probability calculations but before that we need some data cleaning to bring the data in a format that will allow us to extract easily all the information we need. 

We will create another dataset where the the capital letters and punctuations will be ignored. All the unique words in the entire dataset will be seperate columns. Each sms will be a new row and thus each row will tell us which particular word appears how many times in the sms.





In [7]:
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


All punctuations have been removed and all words are lowercased.

## Creating the vocabulary

As stated earlier, we need to create a vocabulary made of unique words to later quantify each sms and traing the computer to recognize a sms with numbers. We will create a vocabulary now.



In [8]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []
for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)

# here we convert the list to a set to keep only the unique words and then convert the set to a list again.        
vocabulary = list(set(vocabulary))

In [9]:
# number of unique words

len(vocabulary)


7783

## The Final Training Set 

We will make a new dataframe. Before that we need to do some pre work. Like, for every sms in the dataset we will make a dictionary which will have lists containing the number of words in each sms. But first, we will initiate the list with **zeroes** 

In [10]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

We loop over training_set['SMS'] using at the same time the enumerate() function to get both the index and the SMS message (index and sms). Here, the enumerate function returns an index number with the word in the sms.

Using a nested loop, we loop over sms (where sms is a list of strings, where each string represents a word in a message).
We incremenent word_counts_per_sms[word][index] by 1.


In [11]:
for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1


By this we actually replaced the **zeroes** in the dictionary 'word_counts_per_sms' with the number of words in the vocabulary.

In [12]:
# Lets just take a look at the first few items in this dict 

import itertools

N = 2
out = dict(itertools.islice(word_counts_per_sms.items(), N))  

print("Dictionary limited by 2 is : " + str(out))  
    
    

Dictionary limited by 2 is : {'listened2the': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [13]:
# Finally create the dataframe 

word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,listened2the,cheered,wn,desparate,kidding,mwahs,wrk,facebook,titles,yr,...,yoville,79,suitemates,6669,shortly,hectic,n,mone,yogasana,round
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Lets get a compiled dataframe adding the above dataframe with the training set

In [14]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,listened2the,cheered,wn,desparate,kidding,mwahs,wrk,facebook,...,yoville,79,suitemates,6669,shortly,hectic,n,mone,yogasana,round
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants First

Naive Bayes algorithm will need to know the probability values of the two equations to be able to classify new messages.

Some of the terms in the equations above will have the same value for every new message. As a start, let's first calculate:

P(Spam) and P(Ham)
NSpam, NHam, NVocabulary

We'll also use Laplace smoothing and set $\alpha = 1$

In [16]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

## Calculating Parameters

P(wi|Spam) and P(wi|Ham) will vary depending on the individual words. For instance, P("secret"|Spam) will have a certain probability value, while P("cousin"|Spam) or P("lovely"|Spam) will most likely have other values.

We will now come down to calculate the probability of a particular word be in spam or ham. For that, we have to calculate two probabilities :

![Image](http://localhost:8888/files/Documents/Building%20A%20Spam%20Filter%20With%20Naive%20Bayes/Two%20prob.svg?_xsrf=2%7Cfe9c1f85%7C545656bfeff32b1d9abf70e25faacf18%7C1597820880)

![Image](http://localhost:8888/files/Documents/Building%20A%20Spam%20Filter%20With%20Naive%20Bayes/p_ham.svg?_xsrf=2%7Cfe9c1f85%7C545656bfeff32b1d9abf70e25faacf18%7C1597820880)

Lets create a for loop that will calculate the equations

In [17]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

## Classifying Messages

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

Takes in as input a new message (w1, w2, ..., wn)
Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

Now , we will use the formulas (calculated by researchers after many experiments and measurements) to classify messages. We will take each words from messages and vocabulary.



In [18]:
import re

def classify(message):
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [19]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [20]:
classify("Sounds good, Tom, then see u there")


P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


আসলে সহজ একটা বিসয়। একটা মেসেজ এর ওয়ার্ড গুলা আলাদা করে স্পাম আর হাম এর শব্দ গুলার সাথে কম্পেয়ার করে দেখা যে অই শব্দ টা স্পাম এ বেশি ছিল নাকি হাম এ । যদি এক টা মেসেজ এর শব্দ গুলার বেশির ভাগ এ স্পাম এর অংশ হয়,তাহলে সেই মেসেজ টা স্পাম মেসেজ।

## Measuring The Spam Filter's Accuracy

Now that we have a model, we need to find how accurate this model works. For that we will use our test set.

In [21]:
def classify_test_set(message):    
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

We will make a new column in the test set that will have predicted values. We will then compare the predicted values with the actual values and then find the accuracy.

In [22]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now, we'll write a function to measure the accuracy of our spam filter to find out how well our spam filter does.

In [26]:
correct = 0
total = test_set.shape[0]
    
for row in test_set.iterrows() : # iterrows returns with column entries in dataset
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


## Not bad !

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set we used, which is a pretty good result. Our initial goal was an accuracy of over 80%, and we managed to do way better than that.

